In [ ]:
import nflreadpy as nfl
import pandas as pd
import polars as pl
import polars.selectors as cs
from g_nfl.modelling.metrics import (
    calculate_havoc,
    calculate_epa_metrics,
    dual_epa_metrics,
)

pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_rows(32)

import jupyter_black

jupyter_black.load()

In [ ]:
WEEK, SEASON = 10, 2025

# SEASONS = list(range(2021, SEASON + 1))
SEASONS = [2018, 2019, 2022, 2023, 2024, 2025]
SEASONS

In [ ]:
pbp: pl.dataframe = nfl.load_pbp(seasons=SEASONS)
pbp.shape

In [ ]:
pbp.sample(5)

In [ ]:
pbp.select(cs.contains("team"))

In [ ]:
pbp.sample(4)

In [ ]:
# pbp.select(pl.col("play_type").value_counts())
pbp["play_type"].value_counts(sort=True)

In [ ]:
useful_cols = [
    "play_id",
    "game_id",
    "season_type",  # filter to REG
    "week",
    "posteam",
    "posteam_type",  # home or away, could be useful for splits
    "defteam",
    # "side_of_field",
    # "yardline_100",
    # "game_date",
    # "quarter_seconds_remaining",
    # "half_seconds_remaining",
    # "game_seconds_remaining",
    # "game_half",
    # "quarter_end",
    # "drive",
    # "sp",
    # "qtr",
    # "down",
    # "goal_to_go",
    # "time",
    # "yrdln",
    # "ydstogo",
    # "ydsnet",
    # "desc",
    "play_type",  # filter to pass / run
    "yards_gained",
    # "shotgun",
    # "no_huddle",
    # "qb_dropback",
    # "qb_kneel",
    # "qb_spike",
    # "qb_scramble",
    # "pass_length",
    # "pass_location",
    "air_yards",
    "yards_after_catch",
    # "run_location",
    # "run_gap",
    # "field_goal_result",
    "kick_distance",
    "extra_point_result",
    "two_point_conv_result",
    "home_timeouts_remaining",
    "away_timeouts_remaining",
    # "timeout",
    # "timeout_team",
    # "td_team",
    # "td_player_name",
    # "td_player_id",
    # "posteam_timeouts_remaining",
    # "defteam_timeouts_remaining",
    # "total_home_score",
    # "total_away_score",
    # "posteam_score",
    # "defteam_score",
    # "score_differential",
    # "posteam_score_post",
    # "defteam_score_post",
    # "score_differential_post",
    # "no_score_prob",
    # "opp_fg_prob",
    # "opp_safety_prob",
    # "opp_td_prob",
    # "fg_prob",
    # "safety_prob",
    # "td_prob",
    # "extra_point_prob",
    # "two_point_conversion_prob",
    # "ep",
    "epa",
    # "total_home_epa",
    # "total_away_epa",
    # "total_home_rush_epa",
    # "total_away_rush_epa",
    # "total_home_pass_epa",
    # "total_away_pass_epa",
    # "air_epa",
    # "yac_epa",
    # "comp_air_epa",
    # "comp_yac_epa",
    # "total_home_comp_air_epa",
    # "total_away_comp_air_epa",
    # "total_home_comp_yac_epa",
    # "total_away_comp_yac_epa",
    # "total_home_raw_air_epa",
    # "total_away_raw_air_epa",
    # "total_home_raw_yac_epa",
    # "total_away_raw_yac_epa",
    "wp",
    "def_wp",
    "home_wp",
    "away_wp",
    "wpa",
    # "vegas_wpa",
    # "vegas_home_wpa",
    # "home_wp_post",
    # "away_wp_post",
    # "vegas_wp",
    # "vegas_home_wp",
    # "total_home_rush_wpa",
    # "total_away_rush_wpa",
    # "total_home_pass_wpa",
    # "total_away_pass_wpa",
    # "air_wpa",
    # "yac_wpa",
    # "comp_air_wpa",
    # "comp_yac_wpa",
    # "total_home_comp_air_wpa",
    # "total_away_comp_air_wpa",
    # "total_home_comp_yac_wpa",
    # "total_away_comp_yac_wpa",
    # "total_home_raw_air_wpa",
    # "total_away_raw_air_wpa",
    # "total_home_raw_yac_wpa",
    # "total_away_raw_yac_wpa",
    # "punt_blocked",
    # "first_down_rush",
    # "first_down_pass",
    # "first_down_penalty",
    # "third_down_converted",
    # "third_down_failed",
    # "fourth_down_converted",
    # "fourth_down_failed",
    # "incomplete_pass",
    # "touchback",
    "interception",
    # "punt_inside_twenty",
    # "punt_in_endzone",
    # "punt_out_of_bounds",
    # "punt_downed",
    # "punt_fair_catch",
    # "kickoff_inside_twenty",
    # "kickoff_in_endzone",
    # "kickoff_out_of_bounds",
    # "kickoff_downed",
    # "kickoff_fair_catch",
    "fumble_forced",
    # "fumble_not_forced",
    # "fumble_out_of_bounds",
    # "solo_tackle",
    # "safety",
    # "penalty", ***
    "tackled_for_loss",
    "fumble_lost",
    # "own_kickoff_recovery",
    # "own_kickoff_recovery_td",
    "qb_hit",
    "rush_attempt",
    "pass_attempt",
    "sack",
    "touchdown",
    "pass_touchdown",
    "rush_touchdown",
    # "return_touchdown",
    # "extra_point_attempt",
    # "two_point_attempt",
    # "field_goal_attempt",
    # "kickoff_attempt",
    # "punt_attempt",
    # "fumble",
    # "complete_pass",
    # "assist_tackle",
    # "lateral_reception",
    # "lateral_rush",
    # "lateral_return",
    # "lateral_recovery",
    # "passer_player_id",
    # "passer_player_name",
    # "passing_yards",
    # "receiver_player_id",
    # "receiver_player_name",
    # "receiving_yards",
    # "rusher_player_id",
    # "rusher_player_name",
    # "rushing_yards",
    # "lateral_receiver_player_id",
    # "lateral_receiver_player_name",
    # "lateral_receiving_yards",
    # "lateral_rusher_player_id",
    # "lateral_rusher_player_name",
    # "lateral_rushing_yards",
    # "lateral_sack_player_id",
    # "lateral_sack_player_name",
    # "interception_player_id",
    # "interception_player_name",
    # "lateral_interception_player_id",
    # "lateral_interception_player_name",
    # "punt_returner_player_id",
    # "punt_returner_player_name",
    # "lateral_punt_returner_player_id",
    # "lateral_punt_returner_player_name",
    # "kickoff_returner_player_name",
    # "kickoff_returner_player_id",
    # "lateral_kickoff_returner_player_id",
    # "lateral_kickoff_returner_player_name",
    # "punter_player_id",
    # "punter_player_name",
    # "kicker_player_name",
    # "kicker_player_id",
    # "own_kickoff_recovery_player_id",
    # "own_kickoff_recovery_player_name",
    # "blocked_player_id",
    # "blocked_player_name",
    # "tackle_for_loss_1_player_id",
    # "tackle_for_loss_1_player_name",
    # "tackle_for_loss_2_player_id",
    # "tackle_for_loss_2_player_name",
    # "qb_hit_1_player_id",
    # "qb_hit_1_player_name",
    # "qb_hit_2_player_id",
    # "qb_hit_2_player_name",
    # "forced_fumble_player_1_team",
    # "forced_fumble_player_1_player_id",
    # "forced_fumble_player_1_player_name",
    # "forced_fumble_player_2_team",
    # "forced_fumble_player_2_player_id",
    # "forced_fumble_player_2_player_name",
    # "solo_tackle_1_team",
    # "solo_tackle_2_team",
    # "solo_tackle_1_player_id",
    # "solo_tackle_2_player_id",
    # "solo_tackle_1_player_name",
    # "solo_tackle_2_player_name",
    # "assist_tackle_1_player_id",
    # "assist_tackle_1_player_name",
    # "assist_tackle_1_team",
    # "assist_tackle_2_player_id",
    # "assist_tackle_2_player_name",
    # "assist_tackle_2_team",
    # "assist_tackle_3_player_id",
    # "assist_tackle_3_player_name",
    # "assist_tackle_3_team",
    # "assist_tackle_4_player_id",
    # "assist_tackle_4_player_name",
    # "assist_tackle_4_team",
    # "tackle_with_assist",
    # "tackle_with_assist_1_player_id",
    # "tackle_with_assist_1_player_name",
    # "tackle_with_assist_1_team",
    # "tackle_with_assist_2_player_id",
    # "tackle_with_assist_2_player_name",
    # "tackle_with_assist_2_team",
    # "pass_defense_1_player_id",
    # "pass_defense_1_player_name",
    # "pass_defense_2_player_id",
    # "pass_defense_2_player_name",
    # "fumbled_1_team",
    # "fumbled_1_player_id",
    # "fumbled_1_player_name",
    # "fumbled_2_player_id",
    # "fumbled_2_player_name",
    # "fumbled_2_team",
    # "fumble_recovery_1_team",
    # "fumble_recovery_1_yards",
    # "fumble_recovery_1_player_id",
    # "fumble_recovery_1_player_name",
    # "fumble_recovery_2_team",
    # "fumble_recovery_2_yards",
    # "fumble_recovery_2_player_id",
    # "fumble_recovery_2_player_name",
    # "sack_player_id",
    # "sack_player_name",
    # "half_sack_1_player_id",
    # "half_sack_1_player_name",
    # "half_sack_2_player_id",
    # "half_sack_2_player_name",
    "return_team",
    "return_yards",
    "penalty_team",
    # "penalty_player_id",
    # "penalty_player_name",
    "penalty_yards",
    # "replay_or_challenge",
    # "replay_or_challenge_result",
    # "penalty_type",
    # "defensive_two_point_attempt",
    # "defensive_two_point_conv",
    # "defensive_extra_point_attempt",
    # "defensive_extra_point_conv",
    # "safety_player_name",
    # "safety_player_id",
    "season",
    "cp",
    "cpoe",
    "series",
    "series_success",
    "series_result",
    # "order_sequence",
    # "start_time",
    # "time_of_day",
    # "stadium",
    # "weather",
    # "nfl_api_id",
    # "play_clock",
    # "play_deleted",
    # "play_type_nfl",
    # "special_teams_play",
    # "st_play_type",
    # "end_clock_time",
    # "end_yard_line",
    # "fixed_drive",
    # "fixed_drive_result",
    # "drive_real_start_time",
    # "drive_play_count",
    # "drive_time_of_possession",
    # "drive_first_downs",
    # "drive_inside20",
    # "drive_ended_with_score",
    # "drive_quarter_start",
    # "drive_quarter_end",
    # "drive_yards_penalized",
    # "drive_start_transition",
    # "drive_end_transition",
    # "drive_game_clock_start",
    # "drive_game_clock_end",
    # "drive_start_yard_line",
    # "drive_end_yard_line",
    # "drive_play_id_started",
    # "drive_play_id_ended",
    # "away_score",
    # "home_score",
    # "location",
    # "result",
    # "total",
    # "spread_line",
    # "total_line",
    # "div_game",
    # "roof",
    # "surface",
    # "temp",
    # "wind",
    # "home_coach", ***
    # "away_coach", ***
    # "stadium_id",
    # "game_stadium",
    # "aborted_play",
    "success",  # success rate
    # "passer",
    # "passer_jersey_number",
    # "rusher",
    # "rusher_jersey_number",
    # "receiver",
    # "receiver_jersey_number",
    "pass",  # filtering
    "rush",  # filtering
    "first_down",  # sum
    # "special",
    # "play",
    "passer_id",
    # "rusher_id",
    # "receiver_id",
    # "name",
    # "jersey_number",
    # "id",
    # "fantasy_player_name",
    # "fantasy_player_id",
    # "fantasy",
    # "fantasy_id",
    # "out_of_bounds",
    # "home_opening_kickoff",
    # "qb_epa",
    # "xyac_epa",
    # "xyac_mean_yardage",
    # "xyac_median_yardage",
    # "xyac_success",
    # "xyac_fd",
    # "xpass",
    "pass_oe",  # avg
]


# features = [
#     "epa",  # avg per play & total
# ]

In [ ]:
pbp.select(useful_cols)

In [ ]:
WP_FILTER = 0.05

pbp_filtered = pbp.select(useful_cols).filter(
    pl.col("play_type").is_in(["run", "pass"])
    & (pl.col("season_type") == "REG")
    & (pl.col("wp") >= WP_FILTER)
    & (pl.col("wp") <= (1 - WP_FILTER))
)
pbp_filtered

In [ ]:
pbp_filtered_aug = pbp_filtered.with_columns(
    pl.any_horizontal(
        ["fumble_forced", "tackled_for_loss", "interception", "sack", "qb_hit"]
    ).alias("havoc"),
    # normalize pass_oe
    (pl.col("pass_oe") / 100).alias("pass_oe"),
    (pl.col("cpoe") / 100).alias("cpoe"),
    # Separate rush and pass yards
    (
        pl.when(pl.col("pass_attempt") == 1)
        .then(pl.col("yards_gained"))
        .otherwise(None)
        .alias("pass_yards")
    ),
    (
        pl.when(pl.col("rush_attempt") == 1)
        .then(pl.col("yards_gained"))
        .otherwise(None)
        .alias("rush_yards")
    ),
    # Explosive play rates
    ((pl.col("yards_gained") >= 15) & (pl.col("play_type") == "pass")).alias(
        "explosive_pass"
    ),
    ((pl.col("yards_gained") >= 10) & (pl.col("play_type") == "run")).alias(
        "explosive_rush"
    ),
    (
        ((pl.col("yards_gained") >= 15) & (pl.col("play_type") == "pass"))
        | ((pl.col("yards_gained") >= 10) & (pl.col("play_type") == "run"))
    ).alias("explosive_play"),
)
pbp_filtered_aug.sample(6)

In [ ]:
id_cols = ["season", "week", "posteam", "defteam"]

In [ ]:
feature_cols = [
    "yards_gained",
    "air_yards",
    "yards_after_catch",
    "epa",
    "interception",
    "fumble_forced",
    "tackled_for_loss",
    "qb_hit",
    "sack",
    "touchdown",
    "cpoe",
    "success",
    "first_down",
    "pass_oe",
    "pass_yards",
    "rush_yards",
    "explosive_pass",
    "explosive_rush",
    "explosive_play",
    "havoc",
]

In [ ]:
pbp_model = pbp_filtered_aug.select(id_cols + feature_cols)
pbp_model.sample(8)

In [ ]:
feature_sum_cols = [
    "yards_gained",
    "yards_after_catch",
    "epa",
    "sack",
    "qb_hit",
    "first_down",
    "touchdown",
    "pass_yards",
    "rush_yards",
    "explosive_pass",
    "explosive_rush",
    "explosive_play",
]
feature_mean_cols = [
    "yards_gained",
    "air_yards",
    "yards_after_catch",
    "epa",
    "interception",
    "fumble_forced",
    "tackled_for_loss",
    "qb_hit",
    "sack",
    "touchdown",
    "cpoe",
    "success",
    "first_down",
    "pass_oe",
    "pass_yards",
    "rush_yards",
    "explosive_pass",
    "explosive_rush",
    "explosive_play",
    "havoc",
]

In [ ]:
agg_exprs = (
    [pl.col("season").count().alias("plays")]
    + [pl.col(c).sum() for c in feature_sum_cols]
    + [pl.col(c).mean().alias(f"{c}_mean") for c in feature_mean_cols]
)
weekly_off = pbp_model.group_by(["season", "week", "posteam"]).agg(agg_exprs)
weekly_off

In [ ]:
agg_exprs = (
    [pl.col("season").count().alias("plays")]
    + [pl.col(c).sum() for c in feature_sum_cols]
    + [pl.col(c).mean().alias(f"{c}_mean") for c in feature_mean_cols]
)
weekly_def = pbp_model.group_by(["season", "week", "defteam"]).agg(agg_exprs)
weekly_def

In [ ]:
weekly_combined = weekly_off.join(
    weekly_def,
    left_on=["season", "week", "posteam"],
    right_on=["season", "week", "defteam"],
    suffix="_def",
)
weekly_combined: pl.DataFrame = weekly_combined.rename({"posteam": "team"}).sort(
    "team", "season", "week"
)
weekly_combined

In [ ]:
sched_df: pl.DataFrame = nfl.load_schedules(seasons=SEASONS).filter(
    pl.col("game_type") == "REG"
)

In [ ]:
ROLLING_WEEKS = 4

# Add rolling sums
df_rolled = weekly_combined.sort("team", "season", "week").with_columns(
    cs.exclude("team", "season", "week")
    .rolling_sum(window_size=ROLLING_WEEKS, min_samples=1)
    .over("team", "season")
    .name.suffix(f"_last_{ROLLING_WEEKS}w")
)

# Add cumulative season sums (not total season sum)
df_rolled = df_rolled.with_columns(
    (cs.exclude("team", "season", "week") - cs.ends_with(f"_last_{ROLLING_WEEKS}w"))
    .cum_sum()
    .over("team", "season")
    .name.suffix("_season")
)

# Fill in missing weeks with forward-fill
# Create all possible team/season/week combinations
all_weeks = sched_df.select("season", "week").unique()
all_teams = df_rolled.select("team").unique()
all_combinations = all_teams.join(all_weeks, how="cross")

# Join and forward-fill
df_rolled = (
    all_combinations.join(df_rolled, on=["team", "season", "week"], how="left")
    .sort("team", "season", "week")
    .with_columns(
        cs.ends_with(("_season", f"_last_{ROLLING_WEEKS}w"))
        .forward_fill()
        .over("team", "season")
    )
)

In [ ]:
df_rolled.filter([pl.col("week") == 5, pl.col("season") == 2025])

In [ ]:
stats = df_rolled.select(
    "team",
    "season",
    "week",
    cs.ends_with("_season"),
    cs.ends_with(f"_last_{ROLLING_WEEKS}w"),
)
away_stats = stats.select(
    pl.col("team").alias("away_team"),
    "season",
    "week",
    cs.ends_with(("_season", f"_last_{ROLLING_WEEKS}w")).name.prefix("away_"),
)

home_stats = stats.select(
    pl.col("team").alias("home_team"),
    "season",
    "week",
    cs.ends_with(("_season", f"_last_{ROLLING_WEEKS}w")).name.prefix("home_"),
)

schedule_with_stats = (
    sched_df.select(
        [
            "game_id",
            "season",
            "week",
            "away_team",
            "home_team",
            "result",
            "spread_line",
        ]
    )
    .join(
        away_stats,
        left_on=["away_team", "season", pl.col("week") - 1],
        right_on=["away_team", "season", "week"],
        how="left",
    )
    .drop("season_right", "week_right", "away_team_right")
    .join(
        home_stats,
        left_on=["home_team", "season", pl.col("week") - 1],
        right_on=["home_team", "season", "week"],
        how="left",
    )
    .drop("season_right", "week_right", "home_team_right")
)

In [ ]:
schedule_with_stats.filter(
    [pl.col("week") == WEEK, pl.col("season") == 2025]
).null_count().sum_horizontal()

In [ ]:
schedule_with_stats.filter([pl.col("week") == WEEK, pl.col("season") == 2025]).head(
    10
).drop(pl.col("result")).null_count().sum_horizontal()

In [ ]:
reduced_schedule_with_stats = schedule_with_stats.filter(
    (pl.col("week") >= 4)
    & (
        (pl.col("season") < 2025)
        | ((pl.col("season") == 2025) & (pl.col("week") <= WEEK))
    )
)
reduced_schedule_with_stats.head()

In [ ]:
reduced_schedule_with_stats.tail()

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# Prepare data
feature_cols = [
    col
    for col in reduced_schedule_with_stats.columns
    if col
    not in [
        "game_id",
        "season",
        "week",
        "away_team",
        "home_team",
        "result",
        "spread_line",
    ]
]

In [ ]:

# df_model = reduced_schedule_with_stats.drop_nulls()

# Convert directly to numpy (faster than pandas)
X = reduced_schedule_with_stats.select(feature_cols).to_numpy()
y = reduced_schedule_with_stats.select("result").to_numpy().ravel()

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
# Train model
model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=4,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    colsample_bylevel=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
)

model.fit(X_train, y_train)

# Predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Evaluate
print("Training Performance:")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")
print(f"  MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")

print("\nTest Performance:")
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")
print(f"  MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")

# Feature importance
import pandas as pd

importance_df = pd.DataFrame(
    {"feature": feature_cols, "importance": model.feature_importances_}
).sort_values("importance", ascending=False)

importance_df.tail(20)

In [ ]:
importance_df

In [ ]:
df_cur_week = reduced_schedule_with_stats.filter(
    (pl.col("week") == WEEK) & (pl.col("season") == SEASON)
)
df_cur_week

In [ ]:
X_cur = df_cur_week.select(feature_cols).to_numpy()

In [ ]:
y_cur = model.predict(X_cur)

In [ ]:
# df_cur_res: pl.DataFrame =
df_cur_week.with_columns(
    [
        pl.lit(y_cur).round(1).alias("model_line"),
    ]
).select(
    [
        "season",
        "week",
        "away_team",
        "home_team",
        "spread_line",
        "model_line",
        (pl.col("model_line") - pl.col("spread_line")).abs().round(1).alias("diff"),
    ]
).sort("diff", descending=True).with_columns(
    [
        pl.when(pl.col("model_line") > pl.col("spread_line"))
        .then(pl.col("home_team"))
        .otherwise(pl.col("away_team"))
        .alias("pick")
    ]
)

In [ ]:
df_cur_week.with_columns(
    [
        pl.lit(y_cur).round(1).alias("model_line"),
    ]
).select(
    [
        "season",
        "week",
        "away_team",
        "home_team",
        "spread_line",
        "model_line",
        (pl.col("model_line") - pl.col("spread_line")).abs().round(1).alias("diff"),
    ]
).sort("diff", descending=True).with_columns(
    [
        pl.when(pl.col("model_line") > pl.col("spread_line"))
        .then(pl.col("home_team"))
        .otherwise(pl.col("away_team"))
        .alias("pick")
    ]
)

In [ ]:
# Add predictions back to the test set
X_train_idx, X_test_idx = train_test_split(
    np.arange(len(X)), test_size=0.2, random_state=42
)

df_test: pl.DataFrame = reduced_schedule_with_stats[X_test_idx].with_columns(
    [pl.lit(y_pred_test).round(1).alias("model_line")]
)

# Compare predictions to spread
eval_df = df_test.select(
    [
        "season",
        "week",
        "away_team",
        "home_team",
        "spread_line",
        "result",
        "model_line",
        (pl.col("model_line") - pl.col("spread_line")).abs().round(2).alias("diff"),
    ]
).with_columns(
    [
        (
            (pl.col("model_line") > pl.col("spread_line"))
            == (pl.col("result") > pl.col("spread_line"))
        ).alias("correct")
    ]
)

print(f"ATS Accuracy: {eval_df.select(pl.col('correct').mean()).item() * 100:.2f}%")
print(f"Correct picks: {eval_df.select(pl.col('correct').sum()).item()}/{len(eval_df)}")

In [ ]:
eval_df.select(
    [
        "away_team",
        "home_team",
        "spread_line",
        "model_line",
        "result",
        "diff",
        "correct",
    ]
).sample(10)

In [ ]:
k = 4
eval_df.filter(pl.col("diff") > k).select(pl.col("correct").mean().round(4)).item()

In [ ]:
for k in range(1, 20):
    threshold_picks = eval_df.filter(pl.col("diff") > k)
    print(
        k,
        len(threshold_picks),
        threshold_picks.select(pl.col("correct").mean().round(4)).item(),
    )